<a href="https://colab.research.google.com/github/EmidioLP/Freecodecamp/blob/main/Notebooks/Machine%20Learning%20with%20Python/RNN_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esse notebook é baseado nas aulas disponibilizadas em: https://www.freecodecamp.org/portuguese/learn/machine-learning-with-python/

Esse notebook é baseado no seguinte guia disponibilizado na documentação do Tensorflow: https://www.tensorflow.org/tutorials/text/text_generator

#Imports e Setup

In [29]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

# Dataset

In [30]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Loading dos dados

In [31]:
#Lê e depois decodifica para o formato py2
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# "Lenght of text" é o número de caracteres contidos nele
print('Lenght of text: {} characters'.format(len(text)))

Lenght of text: 1115394 characters


In [32]:
#Os primeiros 250 caracteres no texto
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



# Codificando

Como o texto não está codificado em números (que é como o modelo de IA precisa receber para entendê-lo), precisa-se fazer isso manualmente.

In [33]:
vocab = sorted(set(text))
# Mapeando caracteres únicos em indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [34]:
print("Texto: ", text[:13])
print("Codificado: ", text_to_int(text[:13]))

Texto:  First Citizen
Codificado:  [18 47 56 57 58  1 15 47 58 47 64 43 52]


Agora precisamos fazer o processo reverso.

In [35]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


# Exemplos de treinamento

In [36]:
seq_lenght = 100
examples_per_epoch = len(text)//(seq_lenght+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [37]:
sequences = char_dataset.batch(seq_lenght+1, drop_remainder=True)

In [38]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [39]:
for x, y in dataset.take(2):
  print("\n\nEXEMPLO\n")
  print("ENTRADA")
  print(int_to_text(x))
  print("\nSAIDA")
  print(int_to_text(y))



EXEMPLO

ENTRADA
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

SAIDA
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXEMPLO

ENTRADA
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

SAIDA
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [40]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Criando o modelo

In [41]:
def build_model (vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                batch_input_shape=[batch_size, None]),
      tf.keras.layers.LSTM(rnn_units,
                           return_sequences=True,
                           stateful=True,
                           recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_1 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


# Função de perca

In [42]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_lenght, vocab_size)")

(64, 100, 65) # (batch_size, sequence_lenght, vocab_size)


In [43]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-6.05892739e-04 -3.37977847e-03 -3.90926888e-03 ...  8.45648465e-04
   -8.91830027e-03 -4.83177463e-03]
  [ 5.10096899e-04 -2.40950333e-03 -8.15306511e-03 ... -1.41341193e-03
   -3.59776616e-03 -6.67714281e-03]
  [-4.77349525e-03 -2.94561265e-04 -1.00216772e-02 ... -5.39078936e-03
   -3.14118853e-03 -3.71610979e-03]
  ...
  [-5.74917858e-03 -7.53174117e-03 -1.00325765e-02 ... -6.27444265e-03
   -1.62619096e-03  4.76791849e-03]
  [-4.45933128e-03 -4.96851932e-03 -5.12435474e-03 ... -2.61492771e-03
   -1.22301513e-03  4.84899245e-03]
  [-4.32439800e-03 -6.97122980e-03 -6.97569409e-03 ... -1.12863490e-03
   -9.89405345e-03 -1.56578771e-03]]

 [[-2.35223700e-03  2.36068107e-03 -1.48341828e-03 ...  1.29843224e-03
    3.83960200e-04 -1.91300781e-03]
  [-2.41667207e-04  8.57054256e-04 -5.26400656e-03 ... -1.76611531e-03
    3.31099238e-03 -4.37120069e-03]
  [-2.18999432e-03  1.60882063e-03 -4.81852237e-03 ...  3.13408091e-05
    2.58739712e-03 -4.95364983e-03]
  ...
  [ 8.938

In [44]:
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[-0.00060589 -0.00337978 -0.00390927 ...  0.00084565 -0.0089183
  -0.00483177]
 [ 0.0005101  -0.0024095  -0.00815307 ... -0.00141341 -0.00359777
  -0.00667714]
 [-0.0047735  -0.00029456 -0.01002168 ... -0.00539079 -0.00314119
  -0.00371611]
 ...
 [-0.00574918 -0.00753174 -0.01003258 ... -0.00627444 -0.00162619
   0.00476792]
 [-0.00445933 -0.00496852 -0.00512435 ... -0.00261493 -0.00122302
   0.00484899]
 [-0.0043244  -0.00697123 -0.00697569 ... -0.00112863 -0.00989405
  -0.00156579]], shape=(100, 65), dtype=float32)


In [45]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[-0.00060589 -0.00337978 -0.00390927 -0.00226954  0.00746787  0.00479183
 -0.00399925 -0.00051312 -0.0009087  -0.00073041  0.00209333  0.00452559
 -0.00021021 -0.00367695 -0.00156662  0.00558799  0.00092511  0.00535244
 -0.00179569 -0.00558419 -0.00215855  0.00475921  0.0008378   0.00485588
  0.00113676  0.00373671  0.00531295 -0.0023434   0.00197843  0.00052371
 -0.00287172 -0.00208545 -0.00698696  0.00303419  0.0046215   0.0059813
 -0.0031674  -0.00265574 -0.00802405  0.00397396 -0.00142589 -0.00099517
 -0.00369721 -0.00206712 -0.00384627  0.00241514  0.00144103  0.00172034
 -0.00044787 -0.00601247 -0.0050223  -0.00041257  0.00138143  0.00254072
  0.00084804  0.0020211  -0.00436445  0.00067724 -0.00416494  0.00259625
 -0.0021704   0.00265137  0.00084565 -0.0089183  -0.00483177], shape=(65,), dtype=float32)


In [46]:
sampled_indices = tf.random.categorical(pred, num_samples=1)

sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars

"FkJyv&R;diWX,dM,tST$d.oAO&V'-k;h kqLiMBoW:tW:Ak&v;i?wOy,UAHOzPjrZS,PGgmgFu:t3Ce3h?Izcl;h'loVvbnSyF$i"

In [47]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Compilando o modelo

In [64]:
model.compile(optimizer='adam', loss=loss)

# Craindo checkpoints

In [62]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_(epoch)")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weigths_only=True)

# Conteúdos extras

https://colah.github.io/posts/2015-08-Understanding-LSTMs/